# Narupa Servers

In this notebook, we explore what goes into a Narupa server, and how to choose a flavour for a given application.

This gets into some of the nuts and bolts of how Narupa works under the hood, including some GRPC details. While understanding GRPC is not necessary, it [will help](grpc.io).

## The Narupa Server Architecture

Narupa uses a client-server model. This means we have some stuff running on a server, and some stuff running on a client, and they talk to eachother. It's exactly how most apps you run on your phone work, and websites:

![Narupa Client Server](./images/narupa_client_server.png)

In Narupa, the server is in change of setting up, managing and running simulations and models. This could be trajectory serving, interactive molecular dynamics, or minimizing structures produced in the builder. 

The client is any application that connects to the server. We generally have two types of client, a VR client for viewing and manipulating simulations in VR, and python clients for experimentation and testing. However, our framework is flexible enough that we could have other types of clients, such as web apps.

### GRPC

The clients and the server talk to eachother using [GRPC](grpc.io). GRPC is an application level communication protocol based on the idea of Remote Procedure Call. A client calls *procedures* (methods) on servers *remotely*, and then the server returns the result at some point in the future. GRPC handles all of the details of sending packets over the network for us, letting us focus on building the application. 

GRPC has a number of features that are good for us: 

* High performance. 
* Ability to do streaming (send lots of data from the server) and bidrectional streaming (send lots of data in both directions between client and server). 
* Works in many languages, including python and C#.

### GRPC Services

In GRPC, we create services that lay out a definition of what a server can do. Combining these services produces an application, for example, the Narupa IMD server uses a TrajectoryService, an IMDService, a MultiplayerService, (and a CommandService and a StateService): 

![Narupa IMD Application](./images/narupa_imd_server.png)

In the above image, we left out the Command service and State service as implicit. 
 
Let's look at the Command Service (see [commands notebook](./commands_and_state.ipynb)), which is used in Narupa to run arbitrary commands. The definition is written in a protobuf file.

```proto
package narupa.protocol.command;

service Command {

    /* Get a list of all the commands available on this service */
    rpc GetCommands (GetCommandsRequest) returns (GetCommandsReply) {}

    /* Runs a command on the service */
    rpc RunCommand (CommandMessage) returns (CommandReply) {}
}

message GetCommandsRequest {

}

message GetCommandsReply{
    repeated CommandMessage commands = 1;
}

message CommandReply {
    google.protobuf.Struct result = 1;
}

message CommandMessage {
    string name = 1;
    google.protobuf.Struct arguments = 2;
}
```


So we've defined a service called `Command`, which has two methods: 

* `RunCommand` - takes a `CommandMessage`, consisting of a `name` and a dictionary-like Struct of `arguments`, and returns a `CommandReply`, which is contains a dictionary of results. 
* `GetCommands` - takes a `GetCommandsRequest`, which is an empty message, and returns as `GetCommandsReply` which is a list (`repeated`) of `CommandMessages`. The `CommandMessage`, in turn, consists of a `name`, which tells us the name of the command, and a Struct (a JSON-style dictionary) of default `arguments`. 


To build a server, we take these service definitions and write the code that actually does what this specification says it should do. Then for clients, we just *call* these functions, knowing that they will produce the results we need. 

## The Server Stack

Narupa servers are just a collection of these GRPC services, with some bells and whistles attached.

The Narupa libraries have a lot of servers lying around:

In [3]:
from narupa.app import NarupaApplicationServer, NarupaFrameApplication, NarupaImdApplication
from narupa.core import NarupaServer, GrpcServer
from narupa.trajectory import FrameServer
from narupa.imd import ImdServer
from narupa.multiplayer import MultiplayerServer

What do these all do? What should we be using?

In what follows, we'll work our way up to those high-level application servers, starting from the bottom of the stack and working our way up.

# TLDR

If you want to send Narupa Frames, e.g. trajectory viewing, you probably want the [NarupaFrameApplication](https://narupa.readthedocs.io/en/latest/python/narupa.app.frame_app.html).

If you want to set up an interactive molecular dynamics simulation, you probably want the [NarupaImdApplication](https://narupa.readthedocs.io/en/latest/python/narupa.app.imd_app.html).

### GRPC Server

It all starts with the Grpc Server, which forms the base of all other servers.

In [4]:
grpc_server = GrpcServer(address='localhost', port=0)

By itself, this server doesn't do much other than set up the underlying [gRPC server](grpc.io) with a few little helpers:

In [5]:
print(grpc_server.__doc__)


    A base class for running GRPC servers that handles the starting and closing
    of the underlying server.

    :param address: The IP address at which to run the server.
    :param port: The port on which to run the server.
    


In [6]:
[x for x in dir(grpc_server) if not x.startswith('_')]

['add_service',
 'address',
 'address_and_port',
 'close',
 'logger',
 'port',
 'server',
 'setup_services']

Mainly, it provides a method to gracefully close, access to the address and port the server is running on, and the ability to set up new gRPC servers. 

If you just want to run a GRPC server with a couple of little helpers, this is the one for you.

In practice, Narupa servers always want commands and state synchronisation, so let's add those. 

In [7]:
from narupa.command import CommandService
from narupa.state.state_service import StateService

command_service = CommandService()
state_service = StateService()

These are the python implementations of the Command and State service. If we add them to the server, it will gain the ability to run commands and synchronise state (see the [commands and state](./commands_and_state.ipynb) notebook).

In [8]:
grpc_server.add_service(command_service)
grpc_server.add_service(state_service)

In [9]:
def hello():
    print('hi!')

command_service.register_command('hello', hello)

Let's check that worked

In [27]:
from narupa.core import NarupaClient

with NarupaClient.insecure_channel(address=grpc_server.address, port=grpc_server.port) as client:
    client.run_command('hello')

_Rendezvous: <_Rendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses"
	debug_error_string = "{"created":"@1581610373.629000000","description":"Failed to pick subchannel","file":"src/core/ext/filters/client_channel/client_channel.cc","file_line":3934,"referenced_errors":[{"created":"@1581610373.629000000","description":"failed to connect to all addresses","file":"src/core/ext/filters/client_channel/lb_policy/pick_first/pick_first.cc","file_line":393,"grpc_status":14}]}"
>

Cool! We've just created a functioning Narupa server! If you wanted to write your own GRPC services, you could add them with the same methodology. 

In [10]:
grpc_server.close()

### Narupa Server

Since we almost always want commands and state synchronisation, we've created the Narupa Server object that does exactly that. 

In [63]:
narupa_server = NarupaServer(address='localhost', port=0)

In [64]:
print(narupa_server.__doc__)


    A base for Narupa gRPC servers. Sets up a gRPC server, and automatically
    attaches a :class:`CommandService` and  :class:`StateService` enabling the running of arbitrary commands
    and synchronisation of state.
    


In [65]:
narupa_server.register_command('narupa_hello', hello)

In [66]:
with NarupaClient.insecure_channel(address=narupa_server.address, port=narupa_server.port) as client:
    client.run_command('narupa_hello')

hi!


That's the Narupa server, it's just a GRPC server with the command and state service added. 

### Frame Server

Most Narupa applications want to transmit some sort of simulation data, i.e. Frames, to clients. For that, we need the frame publishing service. Let's add that to our server:

In [67]:
from narupa.trajectory import FramePublisher
from narupa.trajectory import FrameData

In [68]:
frame_publisher = FramePublisher()

In [69]:
narupa_server.add_service(frame_publisher)

In [70]:
frame = FrameData()
frame.values['hello'] = 'hello'
frame_publisher.send_frame(0, frame)

Let's check that we can connect and receive frames. The `NarupaImdClient` class is a python client that knows how to receive frames.

In [73]:
from narupa.app import NarupaImdClient
import time 

with NarupaImdClient.connect_to_single_server(address=narupa_server.address, port=narupa_server.port) as client:
    client.wait_until_first_frame()
    print(client.first_frame)

values {
  key: "hello"
  value {
    string_value: "hello"
  }
}



In [62]:
narupa_server.close()

This is now a functioning frame server! If we wanted, we could connect to this from VR (if we sent something that looked like a molecule). 

Since this is common functionality, we wrap this in the `FrameServer`. Similarly, we do the same for multiplayer and IMD with the `MultiplayerServer` and `ImdServer`:

In [74]:
from narupa.trajectory import FrameServer

In [75]:
frame_server = FrameServer(address='localhost', port=0)

In [76]:
frame_server.send_frame(0, frame)

In [77]:
with NarupaImdClient.connect_to_single_server(address=narupa_server.address, port=narupa_server.port) as client:
    client.wait_until_first_frame()
    print(client.first_frame)

values {
  key: "hello"
  value {
    string_value: "hello"
  }
}



### Multiplayer and Discovery

This is good, but what about multiplayer? Also, we have to know the address and port to connect to, how do we add discovery? 

We could add the multiplayer service manually:

In [80]:
from narupa.multiplayer.multiplayer_service import MultiplayerService

frame_server.add_service(MultiplayerService())

Now, a client can successfully join the multiplayer, and set values:

In [95]:
with NarupaImdClient.connect_to_single_server(address=frame_server.address, port=frame_server.port) as client:
    client.join_multiplayer('client A')
    client.set_shared_value('a', 2)
    with NarupaImdClient.connect_to_single_server(address=frame_server.address, port=frame_server.port) as second_client:
        second_client.join_multiplayer('client B')
        time.sleep(0.05) # Wait for messages to be received.
        print(second_client.latest_multiplayer_values)

{'a': 2.0}


Likewise, we can manually set up a Discovery server, so our server can be found on the local network. 

In [98]:
from narupa.essd import DiscoveryServer, ServiceHub, DiscoveryClient

In [97]:
discovery_server = DiscoveryServer()

We define the service hub, specifying what services are available and which port they are running at.

In [101]:
service_hub = ServiceHub(name="My Frame Server", address=frame_server.address)
service_hub.add_service("trajectory", frame_server.port)
service_hub.add_service("multiplayer",frame_server.port)

In [102]:
discovery_server.register_service(service_hub)

The discovery server will now be broadcasting the existence of the server! Let's search for it:

In [104]:
discovery_client = DiscoveryClient()

In [109]:
import pprint
pprint.pprint(list(discovery_client.search_for_services(search_time=1.0)))

[ServiceHub(**{'name': 'My Frame Server', 'address': '127.0.0.1', 'id': '373fbf67-207e-492a-9e59-8133e8aa58b9', 'essd_version': '1.0.0', 'services': {'trajectory': 54982, 'multiplayer': 54982}}),
 ServiceHub(**{'name': 'Narupa Tinder Server', 'address': '172.23.43.33', 'port': 38801, 'id': 'a8481099-fdd1-4fa5-9fd4-434e07debd2c', 'essd_version': '1.0.0', 'services': {'multiplayer': 38801, 'trajectory': 38801}})]


You may find some other servers that are running on the network, but hopefully your server was found!

Now we can use the client's autoconnect functionality (note this may not work if you've got multiple servers on the network):

In [111]:
with NarupaImdClient.autoconnect() as client:
    print(client.first_frame)

values {
  key: "hello"
  value {
    string_value: "hello"
  }
}



In [114]:
frame_server.close()

## The Application Servers

Phew, that was quite a lot of work! Luckily, we have a handy wrapper that does all of that for you, the `NarupaApplicationServer`

In [113]:
print(NarupaApplicationServer.__doc__)


    Provides a convenient Narupa server for typical applications, with local
    area network discovery provided by ESSD, multiplayer configuration and a
    command service.

    Use this a base for building specific applications by inheriting from it
    and attaching additional services.
    


The `NarupaFrameApplication` and `NarupaImdApplication` classes inherit from the `NarupaApplicationServer`. The former adds frame support, while the latter adds both frame support and IMD support. Let's try it out.

In [124]:
imd_app = NarupaImdApplication.basic_server(name="My First Narupa Imd App", port=0)

In [125]:
[x for x in dir(imd_app) if not x.startswith('_')]

['DEFAULT_SERVER_NAME',
 'add_service',
 'address',
 'basic_server',
 'close',
 'discovery',
 'frame_publisher',
 'imd',
 'name',
 'port',
 'running_discovery',
 'server']

If you were writing your own interactive molecular dynamics application, you this is all you need. You can send frames, and you'll receive interactions that you can apply to your MD:

In [129]:
imd_app.frame_publisher.send_frame(0, frame)

In [130]:
imd_app.imd.active_interactions

{}

In [ ]:
with NarupaImdClient.autoconnect() as client:
    client.start_interaction()